In [1]:
from helper_classes import Data
import util as ut

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten,Dropout
from tensorflow.keras.regularizers import l2


import warnings
import logging
from scipy import stats
import numpy as np
import random
import torch
from sklearn.preprocessing import MultiLabelBinarizer
random_state = 1
np.random.seed(random_state)
random.seed(random_state)
warnings.filterwarnings('ignore')

In [2]:
dataset='WN18'
kg_root = 'KGs/'+dataset+'/'
dataset=Data(kg_root)

In [3]:
y=[]
x=[]

entitiy_idx=dict()

sub_obj_pairs= dataset.get_entity_pairs_with_predicates(dataset.train_data)
for s_o_pair, predicates in sub_obj_pairs.items():
    s,o=s_o_pair
    entitiy_idx.setdefault(s, len(entitiy_idx))
    entitiy_idx.setdefault(o, len(entitiy_idx))
    x.append([entitiy_idx[s],entitiy_idx[o]])
    y.append(list(predicates))
x=np.array(x)

In [4]:
x=np.array(x)
binarizer=MultiLabelBinarizer()
y=binarizer.fit_transform(y)

In [5]:
embedding_dim=100
model = Sequential()
model.add(Embedding(len(entitiy_idx), embedding_dim, input_length=2,activity_regularizer=l2(0.1)))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(embedding_dim*3, activation='relu',activity_regularizer=l2(0.1)))
model.add(Dropout(.1))
model.add(Dense(y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 100)            4094300   
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               60300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 18)                5418      
Total params: 4,160,018
Trainable params: 4,160,018
Non-trainable params: 0
______________________________________________

In [6]:
history = model.fit(x, y, batch_size=1000, epochs= 50, use_multiprocessing=True,verbose=1,shuffle=True)

Train on 141165 samples
Epoch 1/50
141165/141165 [==============================] - 6s 45us/sample - loss: 2.3466 - accuracy: 0.4078
Epoch 2/50
141165/141165 [==============================] - 6s 40us/sample - loss: 1.3023 - accuracy: 0.7465
Epoch 3/50
141165/141165 [==============================] - 6s 41us/sample - loss: 0.7963 - accuracy: 0.8701
Epoch 4/50
141165/141165 [==============================] - 6s 45us/sample - loss: 0.5909 - accuracy: 0.9184
Epoch 5/50
141165/141165 [==============================] - 6s 41us/sample - loss: 0.4782 - accuracy: 0.9409
Epoch 6/50
141165/141165 [==============================] - 7s 47us/sample - loss: 0.4062 - accuracy: 0.9540
Epoch 7/50
141165/141165 [==============================] - 6s 40us/sample - loss: 0.3560 - accuracy: 0.9603
Epoch 8/50
141165/141165 [==============================] - 7s 47us/sample - loss: 0.3182 - accuracy: 0.9658
Epoch 9/50
141165/141165 [==============================] - 6s 43us/sample - loss: 0.2892 - accuracy: 0.

In [7]:
del x
del y

In [8]:
def evaluation(model,binarizer,dataset,triples):
    x_=[]
    y_=[]
    
    hits = []
    ranks=[]
    for i in range(10):
        hits.append([])

    rank_per_relation=dict()
        
    for i in triples:
        s,p,o=i
        x_.append((entitiy_idx[s],entitiy_idx[o]))
        y_.append(p)
    
    
    tensor_pred=torch.from_numpy(model.predict(np.array(x_)))
    
    _, ranked_predictions =tensor_pred.topk(k=len(binarizer.classes_))
    
    ranked_predictions=ranked_predictions.numpy()
    
    
    assert len(ranked_predictions)==len(y_)
    
    classes_=binarizer.classes_.tolist()
        
    for i in range(len(y_)):
        true_relation=y_[i]
        ith_class=classes_.index(true_relation)
        
        rank = np.where(ranked_predictions[i]==ith_class)[0]
        

        rank_per_relation.setdefault(true_relation, []).append(rank+1)
        
        ranks.append(rank+1)
        
        for hits_level in range(10):
            if rank <= hits_level:
                hits[hits_level].append(1.0)
    

    hits=np.array(hits)
    ranks=np.array(ranks)
    print('########## Relation Prediction Results ##########')

    print('Mean Hits @5: {0}'.format(sum(hits[4]) / (float(len(y_)))))
    print('Mean Hits @3: {0}'.format(sum(hits[2]) / (float(len(y_)))))
    print('Mean Hits @1: {0}'.format(sum(hits[0]) / (float(len(y_)))))
    print('Mean rank: {0}'.format(np.mean(ranks)))
    print('Mean reciprocal rank: {0}'.format(np.mean(1. / ranks)))
    
    print('########## Relation Prediction Analysis ##########')
    
    for pred, ranks in rank_per_relation.items():
        ranks=np.array(ranks)
        
        average_hit_at_1=np.sum(ranks==1)/len(ranks)
        average_hit_at_3=np.sum(ranks<=3)/len(ranks)
        average_hit_at_5=np.sum(ranks<=5)/len(ranks)
        
        print('{0}:\t Hits@1:\t{1:.3f}'.format(pred,average_hit_at_1))
        print('{0}:\t Hits@3:\t{1:.3f}'.format(pred,average_hit_at_3))
        print('{0}:\t Hits@5:\t{1:.3f}'.format(pred,average_hit_at_5))
        print('{0}:\t MRR:\t{1:.3f}\t number of occurrence {2}'.format(pred,np.mean(1. / ranks),len(ranks)))
        print('################################')
    

In [9]:
evaluation(model,binarizer,dataset,dataset.train_data)

########## Relation Prediction Results ##########
Mean Hits @5: 1.0
Mean Hits @3: 0.9999929299642256
Mean Hits @1: 0.997087145260955
Mean rank: 1.0029411348821424
Mean reciprocal rank: 0.9985394484429426
########## Relation Prediction Analysis ##########
_hyponym:	 Hits@1:	0.998
_hyponym:	 Hits@3:	1.000
_hyponym:	 Hits@5:	1.000
_hyponym:	 MRR:	0.999	 number of occurrence 34832
################################
_hypernym:	 Hits@1:	0.998
_hypernym:	 Hits@3:	1.000
_hypernym:	 Hits@5:	1.000
_hypernym:	 MRR:	0.999	 number of occurrence 34796
################################
_member_holonym:	 Hits@1:	0.997
_member_holonym:	 Hits@3:	1.000
_member_holonym:	 Hits@5:	1.000
_member_holonym:	 MRR:	0.998	 number of occurrence 7382
################################
_derivationally_related_form:	 Hits@1:	0.998
_derivationally_related_form:	 Hits@3:	1.000
_derivationally_related_form:	 Hits@5:	1.000
_derivationally_related_form:	 MRR:	0.999	 number of occurrence 29715
################################
_i

In [10]:
evaluation(model,binarizer,dataset,dataset.valid_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9964
Mean Hits @3: 0.9932
Mean Hits @1: 0.9692
Mean rank: 1.0836
Mean reciprocal rank: 0.981476453709689
########## Relation Prediction Analysis ##########
_hypernym:	 Hits@1:	0.969
_hypernym:	 Hits@3:	0.993
_hypernym:	 Hits@5:	0.996
_hypernym:	 MRR:	0.981	 number of occurrence 1174
################################
_derivationally_related_form:	 Hits@1:	0.962
_derivationally_related_form:	 Hits@3:	0.991
_derivationally_related_form:	 Hits@5:	0.996
_derivationally_related_form:	 MRR:	0.977	 number of occurrence 1078
################################
_synset_domain_topic_of:	 Hits@1:	1.000
_synset_domain_topic_of:	 Hits@3:	1.000
_synset_domain_topic_of:	 Hits@5:	1.000
_synset_domain_topic_of:	 MRR:	1.000	 number of occurrence 105
################################
_member_meronym:	 Hits@1:	0.985
_member_meronym:	 Hits@3:	1.000
_member_meronym:	 Hits@5:	1.000
_member_meronym:	 MRR:	0.992	 number of occurrence 273
#############

In [11]:
evaluation(model,binarizer,dataset,dataset.test_data)

########## Relation Prediction Results ##########
Mean Hits @5: 0.9976
Mean Hits @3: 0.9956
Mean Hits @1: 0.9686
Mean rank: 1.0716
Mean reciprocal rank: 0.9815491295306001
########## Relation Prediction Analysis ##########
_member_of_domain_usage:	 Hits@1:	1.000
_member_of_domain_usage:	 Hits@3:	1.000
_member_of_domain_usage:	 Hits@5:	1.000
_member_of_domain_usage:	 MRR:	1.000	 number of occurrence 24
################################
_verb_group:	 Hits@1:	0.923
_verb_group:	 Hits@3:	0.974
_verb_group:	 Hits@5:	1.000
_verb_group:	 MRR:	0.954	 number of occurrence 39
################################
_hyponym:	 Hits@1:	0.973
_hyponym:	 Hits@3:	0.996
_hyponym:	 Hits@5:	0.997
_hyponym:	 MRR:	0.984	 number of occurrence 1153
################################
_member_of_domain_region:	 Hits@1:	0.962
_member_of_domain_region:	 Hits@3:	1.000
_member_of_domain_region:	 Hits@5:	1.000
_member_of_domain_region:	 MRR:	0.981	 number of occurrence 26
################################
_member_meronym:	 H